In [27]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# RMSE 계산 함수 정의
def calculate_rmse(targets, predictions):
    return np.sqrt(mean_squared_error(targets, predictions))

# R2 스코어 계산 함수 정의
def calculate_R2_score(y_test, y_pred):
    return r2_score(y_test, y_pred)

# 데이터 불러오기
input_data = pd.read_csv('2023_smartFarm_AI_hackathon_dataset.csv')

# 데이터 전처리
# 'frmDist' 열을 기준으로 데이터 그룹화 및 평균값 계산하여 'group_df'에 저장
group_df = input_data.groupby('frmDist').mean()

# 'frmDist' 열을 기준으로 데이터 다시 그룹화하고, 'outtrn_cumsum' 열에서 최댓값 계산하여 'group_outtrn_cumsum_df'에 저장
group_outtrn_cumsum_df = input_data.groupby('frmDist').max()[['outtrn_cumsum']]

# 'group_df'에서 'outtrn_cumsum'과 'date' 열 제거
group_df.drop(['outtrn_cumsum', 'date'], axis=1, inplace=True)

# 'group_df'와 'group_outtrn_cumsum_df'를 합쳐 'merge_df' 데이터프레임 생성
merge_df = pd.concat([group_df, group_outtrn_cumsum_df], axis=1)

# 'merge_df'에서 입력 피처(독립 변수)와 타겟 변수(종속 변수) 분리
train_data = merge_df.drop(['outtrn_cumsum', 'HeatingEnergyUsage_cumsum'], axis=1)
target_data = merge_df.loc[:,['outtrn_cumsum', 'HeatingEnergyUsage_cumsum']]

# 데이터 분할: 훈련 데이터와 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(train_data, target_data, random_state=42, test_size=0.2)

# 데이터 표준화
ss = StandardScaler()
ss.fit(X_train)

train_scaled = ss.transform(X_train)
test_scaled = ss.transform(X_test)

# ExtraTreesRegressor 모델 설정 및 하이퍼파라미터 조정
model = ExtraTreesRegressor(
    n_estimators=500,
    max_depth=5,  # 최대 트리 깊이를 제한
    min_samples_split=2,  # 노드를 분할하기 위한 최소 샘플 수
    min_samples_leaf=1,  # 리프 노드에 필요한 최소 샘플 수
    random_state=42
)

# 모델 훈련
model.fit(train_scaled, y_train)

# 모델을 사용하여 테스트 데이터에 대한 예측 수행
y_pred = model.predict(test_scaled)

# RMSE 및 R2 스코어 계산
rmse = calculate_rmse(y_test, y_pred)
r2score = calculate_R2_score(y_test, y_pred)

# 결과 출력
print("RMSE:", rmse)
print("R2_score:", r2score)


RMSE: 14335.79892148121
R2_score: 0.9991312057141644
